<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/callbacks/OpenInferenceCallback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenInference Callback Handler + Arize Phoenix

[OpenInference](https://github.com/Arize-ai/open-inference-spec) is an open standard for capturing and storing AI model inferences. It enables production LLMapp servers to seamlessly integrate with LLM observability solutions such as [Arize](https://arize.com/) and [Phoenix](https://github.com/Arize-ai/phoenix).

The `OpenInferenceCallbackHandler` saves data from LLM applications for downstream analysis and debugging. In particular, it saves the following data in columnar format:

- query IDs
- query text
- query embeddings
- scores (e.g., cosine similarity)
- retrieved document IDs

This tutorial demonstrates the callback handler's use for both in-notebook experimentation and lightweight production logging.

⚠️ The `OpenInferenceCallbackHandler` is in beta and its APIs are subject to change.

ℹ️ If you find that your particular query engine or use-case is not supported, open an issue on [GitHub](https://github.com/Arize-ai/open-inference-spec/issues).

## Configue OpenAI API key

In [ ]:
import os
from getpass import getpass

if os.getenv("OPENAI_API_KEY") is None:
    os.environ["OPENAI_API_KEY"] = getpass(
        "Paste your OpenAI key from:"
        " https://platform.openai.com/account/api-keys\n"
    )
assert os.getenv("OPENAI_API_KEY", "").startswith(
    "sk-"
), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


## Install Dependencies and Import Libraries

Install notebook dependencies.

In [ ]:
%pip install -q html2text llama-index pandas pyarrow tqdm
%pip install -q llama-index-readers-web
%pip install -q llama-index-callbacks-openinference

Import libraries.


In [ ]:
import hashlib
import json
from pathlib import Path
import os
import textwrap
from typing import List, Union

import llama_index.core
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.callbacks import CallbackManager
from llama_index.callbacks.openinference import OpenInferenceCallbackHandler
from llama_index.callbacks.openinference.base import (
    as_dataframe,
    QueryData,
    NodeData,
)
from llama_index.core.node_parser import SimpleNodeParser
import pandas as pd
from tqdm import tqdm

## Load and Parse Documents

Load documents from Paul Graham's essay "What I Worked On".

In [ ]:
documents = SimpleWebPageReader().load_data(
    [
        "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"
    ]
)
print(documents[0].text)



What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in t

Parse the document into nodes. Display the first node's text.

In [ ]:
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)
print(nodes[0].text)

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the

## Access Data as a Pandas Dataframe

When experimenting with chatbots and LLMapps in a notebook, it's often useful to run your chatbot against a small collection of user queries and collect and analyze the data for iterative improvement. The `OpenInferenceCallbackHandler` stores your data in columnar format and provides convenient access to the data as a pandas dataframe.

Instantiate the OpenInference callback handler.

In [ ]:
callback_handler = OpenInferenceCallbackHandler()
callback_manager = CallbackManager([callback_handler])
llama_index.core.Settings.callback_manager = callback_manager

Build the index and instantiate a query engine.

In [ ]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

Run your query engine across a collection of queries.

In [ ]:
max_characters_per_line = 80
queries = [
    "What did Paul Graham do growing up?",
    "When and how did Paul Graham's mother die?",
    "What, in Paul Graham's opinion, is the most distinctive thing about YC?",
    "When and how did Paul Graham meet Jessica Livingston?",
    "What is Bel, and when and where was it written?",
]
for query in queries:
    response = query_engine.query(query)
    print("Query")
    print("=====")
    print(textwrap.fill(query, max_characters_per_line))
    print()
    print("Response")
    print("========")
    print(textwrap.fill(str(response), max_characters_per_line))
    print()

Query
=====
What did Paul Graham do growing up?

Response
Paul Graham grew up writing short stories and programming. He started
programming on an IBM 1401 in 9th grade using an early version of Fortran.
Later, he transitioned to microcomputers and began programming on a TRS-80,
where he wrote simple games and a word processor.

Query
=====
When and how did Paul Graham's mother die?

Response
Paul Graham's mother died when he was 18 years old, from a brain tumor.

Query
=====
What, in Paul Graham's opinion, is the most distinctive thing about YC?

Response
The most distinctive thing about YC, according to Paul Graham, is that instead
of deciding for himself what to work on, the problems come to him. Every 6
months, a new batch of startups brings their problems, which then become the
problems of YC. This aspect of YC's work is engaging and allows for a diverse
range of challenges to be addressed, making it a unique and dynamic environment
for learning about startups.

Query
=====
When an

The data from your query engine runs can be accessed as a pandas dataframe for analysis and iterative improvement.

In [ ]:
query_data_buffer = callback_handler.flush_query_data_buffer()
query_dataframe = as_dataframe(query_data_buffer)
query_dataframe

:id.id:        :timestamp.iso_8601:  \
0  c0ac90c2-706d-41f6-b840-cfff2d5406ce  2024-02-20T16:03:47.852685   
1  234e0990-6ff8-4253-a361-c139871565f4  2024-02-20T16:03:51.083002   
2  4128c740-ba0d-4f44-8c11-4360d6e27137  2024-02-20T16:03:53.847162   
3  2d78599f-af33-4d62-9c24-3f8f95821abf  2024-02-20T16:03:57.576285   
4  7b9abfb5-3687-48bf-9da7-3361e7b401be  2024-02-20T16:03:58.605381   

                                :feature.text:prompt  \
0                What did Paul Graham do growing up?   
1         When and how did Paul Graham's mother die?   
2  What, in Paul Graham's opinion, is the most di...   
3  When and how did Paul Graham meet Jessica Livi...   
4    What is Bel, and when and where was it written?   

                   :feature.[float].embedding:prompt :feature.text:llm_prompt  \
0  [0.00727177644148469, -0.009682492353022099, 0...                     None   
1  [0.015593511052429676, 0.004450097680091858, -...                     None   
2  [0.0027695773169398308, 0.001457934849895537, ...                     None   
3  [0.002315425779670477, -0.0024678888730704784,...                     None   
4  [0.009047380648553371, -0.013641595840454102, ...                     None   

                       :feature.[[str]]:llm_messages  \
0  [(system, You are an expert Q&A system that is...   
1  [(system, You are an expert Q&A system that is...   
2  [(system, You are an expert Q&A system that is...   
3  [(system, You are an expert Q&A system that is...   
4  [(system, You are an expert Q&A system that is...   

                           :prediction.text:response  \
0  Paul Graham grew up writing short stories and ...   
1  Paul Graham's mother died when he was 18 years...   
2  The most distinctive thing about YC, according...   
3  Paul Graham met Jessica Livingston at a big pa...   
4  Bel is a new Lisp that was written in Arc. It ...   

        :feature.[str].retrieved_document_ids:prompt  \
0  [6c2625857520cae8185c229df96c5c8324d998503c98a...   
1  [6c2625857520cae8185c229df96c5c8324d998503c98a...   
2  [9aaeaab1077c1a7faf97ded69d2d037e18ed9d3cb0e63...   
3  [6c2625857520cae8185c229df96c5c8324d998503c98a...   
4  [a4569bc4d16179f406798904d9cc4bbd7d3c0caca3161...   

  :feature.[float].retrieved_document_scores:prompt  
0           [0.808843861957992, 0.7996330023661674]  
1          [0.7773106216458116, 0.7698260744207294]  
2          [0.8299917416870926, 0.8223302097228329]  
3          [0.8240769796488667, 0.8076047627810241]  
4          [0.8315868190669687, 0.7641419929089962]

The dataframe column names conform to the OpenInference spec, which specifies the category, data type, and intent of each column.

## Log Production Data

In a production setting, LlamaIndex application maintainers can log the data generated by their system by implementing and passing a custom `callback` to `OpenInferenceCallbackHandler`. The callback is of type `Callable[List[QueryData]]` that accepts a buffer of query data from the `OpenInferenceCallbackHandler`, persists the data (e.g., by uploading to cloud storage or sending to a data ingestion service), and flushes the buffer after data is persisted. A reference implementation is included below that periodically writes data in OpenInference format to local Parquet files when the buffer exceeds a certain size.

In [ ]:
class ParquetCallback:
    def __init__(
        self, data_path: Union[str, Path], max_buffer_length: int = 1000
    ):
        self._data_path = Path(data_path)
        self._data_path.mkdir(parents=True, exist_ok=False)
        self._max_buffer_length = max_buffer_length
        self._batch_index = 0

    def __call__(
        self,
        query_data_buffer: List[QueryData],
        node_data_buffer: List[NodeData],
    ) -> None:
        if len(query_data_buffer) >= self._max_buffer_length:
            query_dataframe = as_dataframe(query_data_buffer)
            file_path = self._data_path / f"log-{self._batch_index}.parquet"
            query_dataframe.to_parquet(file_path)
            self._batch_index += 1
            query_data_buffer.clear()  # ⚠️ clear the buffer or it will keep growing forever!
            node_data_buffer.clear()  # didn't log node_data_buffer, but still need to clear it

⚠️ In a production setting, it's important to clear the buffer, otherwise, the callback handler will indefinitely accumulate data in memory and eventually cause your system to crash.

Attach the Parquet writer to your callback and re-run the query engine. The data will be saved to disk.

In [ ]:
data_path = "data"
parquet_writer = ParquetCallback(
    data_path=data_path,
    # this parameter is set artificially low for demonstration purposes
    # to force a flush to disk, in practice it would be much larger
    max_buffer_length=1,
)
callback_handler = OpenInferenceCallbackHandler(callback=parquet_writer)
callback_manager = CallbackManager([callback_handler])
llama_index.core.Settings.callback_manager = callback_manager

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

for query in tqdm(queries):
    query_engine.query(query)

100%|██████████| 5/5 [00:13<00:00,  2.70s/it]


Load and display saved Parquet data from disk to verify that the logger is working. 

In [ ]:
query_dataframes = []
for file_name in os.listdir(data_path):
    file_path = os.path.join(data_path, file_name)
    query_dataframes.append(pd.read_parquet(file_path))
query_dataframe = pd.concat(query_dataframes)
query_dataframe

:id.id:        :timestamp.iso_8601:  \
0  e6f01e8e-f774-45da-beff-4fcb6c85726a  2024-02-20T16:04:02.364860   
0  e024727e-4264-4f2b-8ab2-147df11da38d  2024-02-20T16:04:05.025023   
0  75cd0ccd-4899-4055-a8b9-38b4300070e9  2024-02-20T16:04:08.291817   
0  44a0e668-e790-4a4b-ac62-4ee6b545ca6a  2024-02-20T16:04:11.318325   
0  c59da7f0-0f6c-43f1-9600-6ade61cfd658  2024-02-20T16:04:13.206596   

                                :feature.text:prompt  \
0                What did Paul Graham do growing up?   
0         When and how did Paul Graham's mother die?   
0  What, in Paul Graham's opinion, is the most di...   
0  When and how did Paul Graham meet Jessica Livi...   
0    What is Bel, and when and where was it written?   

                   :feature.[float].embedding:prompt :feature.text:llm_prompt  \
0  [0.00727177644148469, -0.009682492353022099, 0...                     None   
0  [0.015593511052429676, 0.004450097680091858, -...                     None   
0  [0.0027695773169398308, 0.001457934849895537, ...                     None   
0  [0.002315425779670477, -0.0024678888730704784,...                     None   
0  [0.009047380648553371, -0.013641595840454102, ...                     None   

                       :feature.[[str]]:llm_messages  \
0  [[system, You are an expert Q&A system that is...   
0  [[system, You are an expert Q&A system that is...   
0  [[system, You are an expert Q&A system that is...   
0  [[system, You are an expert Q&A system that is...   
0  [[system, You are an expert Q&A system that is...   

                           :prediction.text:response  \
0  Growing up, Paul Graham worked on writing shor...   
0  Paul Graham's mother died when he was 18 years...   
0  The most distinctive thing about YC, according...   
0  Paul Graham met Jessica Livingston at a big pa...   
0  Bel is a new Lisp that was written in Arc. It ...   

        :feature.[str].retrieved_document_ids:prompt  \
0  [6c2625857520cae8185c229df96c5c8324d998503c98a...   
0  [6c2625857520cae8185c229df96c5c8324d998503c98a...   
0  [9aaeaab1077c1a7faf97ded69d2d037e18ed9d3cb0e63...   
0  [6c2625857520cae8185c229df96c5c8324d998503c98a...   
0  [a4569bc4d16179f406798904d9cc4bbd7d3c0caca3161...   

  :feature.[float].retrieved_document_scores:prompt  
0           [0.808843861957992, 0.7996330023661674]  
0          [0.7773106216458116, 0.7698260744207294]  
0          [0.8299917416870926, 0.8223302097228329]  
0          [0.8240769796488667, 0.8076047627810241]  
0          [0.8315868190669687, 0.7641419929089962]